In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [16]:
url_inicial = 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/archive'
url_root = 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/archive'
r=requests.get(url_inicial)

In [17]:
soup = BeautifulSoup(r.text, 'html.parser')

In [18]:
box = soup.find('div', class_='issues media-list')
volumen=box.findAll('div', class_='issue-summary media')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/view/687',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/view/676',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/view/664',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/view/661',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/view/659',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/view/653',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/view/650',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/view/645',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/view/626',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/view/605',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/view/614',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/view/598',
 'https://revist

In [19]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('section', class_='section') 
    volumen1=box1.findAll('div', class_='article-summary media')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [20]:
vol2

['https://revistas.juridicas.unam.mx/index.php/derecho-comparado/article/view/17208',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/article/view/16493',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/article/view/15982',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/article/view/15810',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/article/view/15641',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/article/view/15242',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/article/view/15173',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/article/view/14966',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/article/view/14159',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/article/view/13257',
 'https://revistas.juridicas.unam.mx/index.php/derecho-comparado/article/view/13258',
 'https://revistas.juridicas.unam.mx/index.php/derecho

In [21]:
def get_url_items(sopa,url):
    box1 = soup1.find('section', class_='section') 
    volumen1=box1.findAll('div', class_='article-summary media')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [22]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://revistas.juridicas.unam.mx/index.php/derecho-comparado/issue/archive


In [23]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

304

In [24]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [25]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Boletín Mexicano de Derecho Comparado'
    a='V. Ciencias Sociales'
    tem='Derecho'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page-header').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='article-abstract').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', id='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='galley-link btn btn-primary pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [26]:
list_scraper=list_scraper[0:304]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [27]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Boletín Mexicano de Derecho Comparado,V. Ciencias Sociales,Derecho,Créditos editoriales,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
1,Boletín Mexicano de Derecho Comparado,V. Ciencias Sociales,Derecho,Revista completa,,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
2,Boletín Mexicano de Derecho Comparado,V. Ciencias Sociales,Derecho,Revista completa,,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
3,Boletín Mexicano de Derecho Comparado,V. Ciencias Sociales,Derecho,Revista completa,,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
4,Boletín Mexicano de Derecho Comparado,V. Ciencias Sociales,Derecho,Revista completa,,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
...,...,...,...,...,...,...,...,...
299,Boletín Mexicano de Derecho Comparado,V. Ciencias Sociales,Derecho,Preliminares,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
300,Boletín Mexicano de Derecho Comparado,V. Ciencias Sociales,Derecho,Preliminares,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
301,Boletín Mexicano de Derecho Comparado,V. Ciencias Sociales,Derecho,Preliminares,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...
302,Boletín Mexicano de Derecho Comparado,V. Ciencias Sociales,Derecho,Presentación,None,None,https://revistas.juridicas.unam.mx/index.php/d...,https://revistas.juridicas.unam.mx/index.php/d...


In [28]:
df_catalogo.to_csv('Revista79.csv', index=False)